In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from catboost import CatBoostRegressor
import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('data/raw/train.csv', parse_dates=['Date'])
sample_submission = pd.read_csv('data/raw/sample_submission.csv')

In [4]:
train['Date'] = pd.to_datetime(train['Date'].apply(lambda x: x[:-2] + '01'), format='%Y-%m-%d')
train['Month'] = train['Date'].dt.to_period('M').astype(str)

In [5]:
monthly_agg = (
    train
    .groupby(['Company_ID', 'Product_ID', 'Month'], as_index=False)
    .agg({'Target': 'sum'})
    .rename(columns={'Target': 'sum_sales'})
    .sort_values(by=['Company_ID', 'Product_ID', 'Month'])
)

In [6]:
d = {'Company_ID': [], 'Product_ID': [], 'Month': []}
ar = monthly_agg['Company_ID'].sort_values().unique()
br = monthly_agg['Product_ID'].sort_values().unique()
cr = monthly_agg['Month'].sort_values().unique()
for company_id in ar:
    for product_id in br:
        for month in cr:
            d['Company_ID'].append(company_id)
            d['Product_ID'].append(product_id)
            d['Month'].append(month)
d = pd.DataFrame(d)
monthly_agg = d.merge(monthly_agg, 'left', on=['Company_ID', 'Product_ID', 'Month']).fillna(0)

In [7]:
monthly_agg_saved = monthly_agg.copy()
monthly_agg

,Company_ID,Product_ID,Month,sum_sales
0,0,0,2019-01,0.0
1,0,0,2019-02,0.0
2,0,0,2019-03,0.0
3,0,0,2019-04,0.0
4,0,0,2019-05,0.0
...,...,...,...,...
3459115,3,14668,2023-08,0.0
3459116,3,14668,2023-09,0.0
3459117,3,14668,2023-10,0.0
3459118,3,14668,2023-11,0.0


In [8]:
monthly_agg['Month_date'] = pd.to_datetime(monthly_agg['Month'], format='%Y-%m')

n_lags = 3

subset = []
gb = monthly_agg.groupby(['Company_ID','Product_ID'])
for i in range(n_lags + 1):
    monthly_agg[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)
    subset.append(f'prev_sum_sales_{i}')

monthly_agg['target'] = gb['sum_sales'].shift(-1)

In [9]:
monthly_agg.dropna(subset=['target'], inplace=True)

In [10]:
monthly_agg

,Company_ID,Product_ID,Month,sum_sales,Month_date,prev_sum_sales_0,prev_sum_sales_1,prev_sum_sales_2,prev_sum_sales_3,target
0,0,0,2019-01,0.0,2019-01-01,0.0,NaN,NaN,NaN,0.0
1,0,0,2019-02,0.0,2019-02-01,0.0,0.0,NaN,NaN,0.0
2,0,0,2019-03,0.0,2019-03-01,0.0,0.0,0.0,NaN,0.0
3,0,0,2019-04,0.0,2019-04-01,0.0,0.0,0.0,0.0,0.0
4,0,0,2019-05,0.0,2019-05-01,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
3459114,3,14668,2023-07,0.0,2023-07-01,0.0,0.0,5.0,0.0,0.0
3459115,3,14668,2023-08,0.0,2023-08-01,0.0,0.0,0.0,5.0,0.0
3459116,3,14668,2023-09,0.0,2023-09-01,0.0,0.0,0.0,0.0,0.0
3459117,3,14668,2023-10,0.0,2023-10-01,0.0,0.0,0.0,0.0,0.0


In [11]:
# Сортируем по Month_date (по возрастанию времени)
monthly_agg.sort_values(by=['Month_date'], inplace=True)

# Пример: берём валидацию на последние 3 месяца, которые у нас есть в данных
all_months = sorted(monthly_agg['Month_date'].unique())
N_valid_months = 12
valid_threshold = all_months[-N_valid_months]  # первая из последних 3
train_data = monthly_agg[monthly_agg['Month_date'] < valid_threshold]
valid_data = monthly_agg[monthly_agg['Month_date'] >= valid_threshold]

In [12]:
feature_cols = subset
cat_cols = ['Company_ID', 'Product_ID']

X_train = train_data[train_data['target'] != 0][feature_cols + cat_cols]
y_train = train_data[train_data['target'] != 0]['target']

X_valid = valid_data[valid_data['target'] != 0][feature_cols + cat_cols]
y_valid = valid_data[valid_data['target'] != 0]['target']

# Укажем, какие столбцы считать категориальными для CatBoost
# (CatBoost умеет работать с ними напрямую, выучивая таргет-кодирование)
cat_features_indices = [X_train.columns.get_loc(c) for c in cat_cols]  # индексы категориальных фич

In [13]:
X_train

,prev_sum_sales_0,prev_sum_sales_1,prev_sum_sales_2,prev_sum_sales_3,Company_ID,Product_ID
168600,1.0,NaN,NaN,NaN,0,2856
2148720,1.0,NaN,NaN,NaN,2,7101
2148840,1.0,NaN,NaN,NaN,2,7103
2148900,2.0,NaN,NaN,NaN,2,7104
168540,0.0,NaN,NaN,NaN,0,2855
...,...,...,...,...,...,...
1836046,62.0,132.0,127.0,24.0,2,1806
3115606,0.0,1.0,2.0,0.0,3,8844
3131566,4.0,4.0,2.0,6.0,3,9114
1765006,1.0,2.0,5.0,1.0,2,599


In [14]:
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.005,
    depth=8,
    random_seed=42,
    cat_features=cat_features_indices,
    eval_metric='MAE',
    verbose=100
)

model.fit(
    X_train,
    y_train,
    eval_set=(X_valid, y_valid),
    early_stopping_rounds=100
)


0:	learn: 67.6463420	test: 68.2532697	best: 68.2532697 (0)	total: 410ms	remaining: 6m 50s
100:	learn: 47.0974255	test: 47.8905459	best: 47.8905459 (100)	total: 21.7s	remaining: 3m 13s
200:	learn: 36.5908152	test: 37.6209750	best: 37.6209750 (200)	total: 43s	remaining: 2m 50s
300:	learn: 31.2234471	test: 32.4740647	best: 32.4740647 (300)	total: 1m 4s	remaining: 2m 30s
400:	learn: 28.3554609	test: 29.7386227	best: 29.7386227 (400)	total: 1m 26s	remaining: 2m 8s
500:	learn: 26.7250777	test: 28.3870867	best: 28.3870867 (500)	total: 1m 43s	remaining: 1m 42s
600:	learn: 25.6771049	test: 27.6470262	best: 27.6470262 (600)	total: 1m 55s	remaining: 1m 16s
700:	learn: 24.8042080	test: 27.0282653	best: 27.0282653 (700)	total: 2m 2s	remaining: 52.2s
800:	learn: 24.2937008	test: 26.7401567	best: 26.7401567 (800)	total: 2m 12s	remaining: 32.9s
900:	learn: 23.9944754	test: 26.6566887	best: 26.6566887 (900)	total: 2m 22s	remaining: 15.6s
999:	learn: 23.7982245	test: 26.6026905	best: 26.6022475 (983)	to

In [15]:
from sklearn.metrics import mean_absolute_error
y_pred_valid = model.predict(X_valid)
mae_valid = mean_absolute_error(y_valid, y_pred_valid)
print("MAE на валидации:", mae_valid)

MAE на валидации: 26.60224853941893


In [16]:
full_train = monthly_agg.dropna(subset=['target'])
X_full = full_train[full_train['target'] != 0][feature_cols + cat_cols]
y_full = full_train[full_train['target'] != 0]['target']

final_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.005,
    depth=8,
    random_seed=42,
    cat_features=cat_features_indices,
    eval_metric='MAE',
    verbose=100
)

final_model.fit(
    X_full,
    y_full
)

0:	learn: 67.9104568	total: 84.8ms	remaining: 1m 24s
100:	learn: 47.3754088	total: 18.5s	remaining: 2m 45s
200:	learn: 36.8458375	total: 33.8s	remaining: 2m 14s
300:	learn: 31.5053874	total: 54.8s	remaining: 2m 7s
400:	learn: 28.6458201	total: 1m 12s	remaining: 1m 47s
500:	learn: 27.0954495	total: 1m 28s	remaining: 1m 28s
600:	learn: 26.1382337	total: 1m 43s	remaining: 1m 8s
700:	learn: 25.3061146	total: 1m 56s	remaining: 49.5s
800:	learn: 24.7815018	total: 2m 5s	remaining: 31.2s
900:	learn: 24.4506566	total: 2m 16s	remaining: 15s
999:	learn: 24.2257828	total: 2m 27s	remaining: 0us


In [17]:
def predict_next_month(df):
    df = df.copy()
    df['Month_date'] = pd.to_datetime(df['Month'], format='%Y-%m')
    df['Month_pred'] = (df['Month_date'] + pd.offsets.MonthBegin(1)).dt.to_period('M').astype(str)
    df = df.sort_values(by='Month_date')

    gb = df.groupby(['Company_ID','Product_ID'])
    for i in range(n_lags + 1):
        df[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)

    df['Target'] = gb['sum_sales'].shift(-1)

    to_predict = df['Target'].isna()
    df.loc[to_predict, 'Target'] = model.predict(df.loc[to_predict, feature_cols + cat_cols])
    return df.loc[to_predict, ['Company_ID', 'Product_ID', 'Month_pred', 'Target']]

In [18]:
sample_submission['Company_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[0]).astype(int)
sample_submission['Product_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[1]).astype(int)
sample_submission['Month_pred'] = sample_submission['Id'].apply(lambda x: x.split('_')[2]).astype(str)
sample_submission

,Id,Target,Company_ID,Product_ID,Month_pred
0,0_1_2024-01,0,0,1,2024-01
1,0_2_2024-01,0,0,2,2024-01
2,0_3_2024-01,0,0,3,2024-01
3,0_5_2024-01,0,0,5,2024-01
4,0_6_2024-01,0,0,6,2024-01
...,...,...,...,...,...
113851,3_14664_2024-03,0,3,14664,2024-03
113852,3_14665_2024-03,0,3,14665,2024-03
113853,3_14666_2024-03,0,3,14666,2024-03
113854,3_14667_2024-03,0,3,14667,2024-03


In [19]:
preds = []
mag = monthly_agg_saved.copy()
for i in range(3):
    p = predict_next_month(mag)
    preds.append(p.copy())

    p.columns = ['Company_ID', 'Product_ID', 'Month', 'sum_sales']
    mag = pd.concat([mag, p], axis=0)
preds = pd.concat(preds, axis=0)
sample_submission = sample_submission.drop(columns=['Target'], axis=1).merge(preds, 'left', on=['Company_ID', 'Product_ID', 'Month_pred'])

In [20]:
to_mult = monthly_agg_saved[monthly_agg_saved['Month'] == '2023-12'].groupby(['Company_ID', 'Product_ID'])['sum_sales'].agg(lambda x: int(x.sum() > 0)).reset_index()
for i in tqdm.tqdm(to_mult.index):
    sample_submission.loc[(sample_submission['Company_ID'] == to_mult.loc[i, 'Company_ID']) &
                          (sample_submission['Product_ID'] == to_mult.loc[i, 'Product_ID']), 'Target'] *= to_mult.loc[i, 'sum_sales']

100%|██████████| 57652/57652 [00:34<00:00, 1665.43it/s]


In [21]:
sample_submission[['Id', 'Target']]

,Id,Target
0,0_1_2024-01,8.780180
1,0_2_2024-01,0.000000
2,0_3_2024-01,0.000000
3,0_5_2024-01,4.062059
4,0_6_2024-01,4.562599
...,...,...
113851,3_14664_2024-03,0.000000
113852,3_14665_2024-03,224.196244
113853,3_14666_2024-03,0.000000
113854,3_14667_2024-03,27.142775


In [22]:
sample_submission[['Id', 'Target']].to_csv('data/submissions/bruh.csv', index=False)